In [2]:
import sys
sys.path.append("/home/jarlehti/projects/gradu")

In [3]:
import os
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import d3p
from src.napsu_mq.napsu_mq import NapsuMQModel

In [4]:
CURRENT_FOLDER = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
DATASETS_FOLDER = os.path.join(CURRENT_FOLDER, "data", "datasets")
MODELS = os.path.join(CURRENT_FOLDER, "models")

In [5]:
rng = jax.random.PRNGKey(12903863272)

In [6]:
dataset = pd.read_csv(os.path.join(DATASETS_FOLDER, "binary4d.csv"))
n, d = dataset.shape
print(n)
print(d)

100000
4


In [7]:
column_feature_set = [
    ('A', 'D'), 
    ('B', 'D'), 
    ('C', 'D'), 
]

model = NapsuMQModel()
result = model.fit(
    data=dataset,
    dataset_name="binary4d",
    rng=rng,
    epsilon=50,
    delta=(n ** (-2)),
    column_feature_set=column_feature_set,
    use_laplace_approximation=False
)

No experiment_id found: <ContextVar name='experiment_id' at 0x7f1e2e549450>
Setting experiment_id to NCWZWXE0
Recording:  Query selection


/home/jarlehti/projects/gradu/src/napsu_mq/mst.py:120: RuntimeWarning: divide by zero encountered in double_scalars
  epsilon = np.sqrt(8 * rho / (r - 1))


Len weights: 6
Weights: {('A', 'B'): 188.17864210799598, ('A', 'C'): 46.150756576451386, ('A', 'D'): 15549.525653755876, ('B', 'C'): 309.1797743744319, ('B', 'D'): 57.80264887254998, ('C', 'D'): 30081.92132081942}
Sorted weights: [(('A', 'C'), 46.150756576451386), (('B', 'D'), 57.80264887254998), (('A', 'B'), 188.17864210799598), (('B', 'C'), 309.1797743744319), (('A', 'D'), 15549.525653755876), (('C', 'D'), 30081.92132081942)]
Cliques: [('A', 'D'), ('B', 'D'), ('C', 'D')]
T edges: [('A', 'D'), ('B', 'D'), ('C', 'D')]
Recording:  Calculating full marginal query
Recording:  Calculating canonical query set


  0%|                                                                                                                  | 0/8 [00:00<?, ?it/s]
4it [00:00, 7019.76it/s]

2it [00:00, 15363.75it/s]

4it [00:00, 19152.07it/s]

4it [00:00, 11966.63it/s]

2it [00:00, 15279.80it/s]

2it [00:00, 8858.09it/s]

2it [00:00, 18157.16it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 25614.07it/s]

Canonical queries: 7


Recording:  MCMC

                mean       std    median      5.0%     95.0%     n_eff     r_hat
lambdas[0]     -0.01      0.01     -0.01     -0.02      0.01   7962.30      1.00
lambdas[1]     -0.01      0.01     -0.01     -0.03      0.01   5946.80      1.00
lambdas[2]      0.28      0.01      0.28      0.26      0.30   3211.01      1.00
lambdas[3]     -0.69      0.01     -0.69     -0.71     -0.67   6223.47      1.00
lambdas[4]      0.08      0.01      0.08      0.06      0.10   3393.74      1.00
lambdas[5]      0.39      0.01      0.39      0.38      0.40   7703.61      1.00
lambdas[6]     -1.53      0.02     -1.53     -1.56     -1.51   7691.18      1.00

Number of divergences: 0
[lambdas]	 max r_hat: 0.8699
Potential energy	min: [61.67584229  0.          0.          0.        ]	max: [75.05485535  0.          0.          0.        ]	mean: [64.849823  0.        0.        0.      ]	std: [1.89533806 0.         0.         0.        ]
Acceptance probability	min: [0.12866236 0.         0.

In [8]:
result_path = os.path.join(MODELS, "napsu_testing_binary4d_all_queries_500e_NUTS.dill")
result.store(result_path)